In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from __future__ import print_function 
from __future__ import division

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import numpy as np
    import torchvision
    import matplotlib.pyplot as plt
    import time
    import os
    import copy
    from torchvision import datasets, models, transforms
    print("PyTorch Version: ",torch.__version__)
    print("Torchvision Version: ",torchvision.__version__)
except Exception as error:
    !pip install {str(error)[17:-1]}
except:
    print("Pacote não encontrado")

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
import torchvision

torchvision.models.resnet152()

rm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (7): Bottleneck(
      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, aff